In [8]:
# import sqlite3
import pandas as pd
from operator import itemgetter

from langchain_ollama import ChatOllama
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
from langchain.chains import create_sql_query_chain
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from sqlalchemy import create_engine

import mysql.connector

In [2]:
# ### local db create

# data = pd.read_csv('employee_attrition_data.csv')
# conn = sqlite3.connect("employee_records.db")

# # Write the DataFrames to the database as tables
# data.to_sql("employee_master", conn, if_exists="replace", index=False)

# conn.close()



In [3]:
username = "root"
password = "Debasis#1"
database = "employeedb"
host = "localhost"  
port = 3306

In [9]:

llm = ChatOllama(model='llama3:instruct', temperature=0.0)
db = SQLDatabase.from_uri(f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}")
engine = create_engine(f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}")


table_info = db.get_table_info()

In [5]:
# table_info

In [11]:


template_str = f"""
You are an expert SQL assistant.

Your task is to take a user's natural language question and return only a syntactically correct SQL query using ONLY the table and column names exactly as shown.

Use the schema below:
{table_info}

## Example:
Question: Find employees with above-average performance rating but below-average income.
SQL Query: SELECT * FROM employee_master WHERE "PerformanceRating" > (SELECT AVG("PerformanceRating") FROM employee_master) AND "MonthlyIncome" < (SELECT AVG("MonthlyIncome") FROM employee_master);

## Rules:
- Use only the tables/columns from the schema.
- Return ONLY the SQL query (no markdown, no extra text, no formatting).
- Do not need Here is the SQL query to answer this question:
- Do not need to say I am an AI language model.
- Only return the SQL query without any '''


Question: {{question}}
SQL Query: {{query}}
"""

answer_prompt = PromptTemplate.from_template(template_str)

execute_query = QuerySQLDatabaseTool(db=db)
write_query = create_sql_query_chain(llm, db)


chain = (
    RunnablePassthrough.assign(query=write_query)
    .assign(result=itemgetter("query") | execute_query)
    .assign(question=itemgetter("question")) 
    | answer_prompt
    | llm
    | StrOutputParser()
)


sql_query = chain.invoke({"question": "List employees who worked overtime today."})
print(sql_query)


SELECT `per_no`, `sec_cd` FROM employeerecord WHERE `ota_hr` > 0 AND `shft_date` = CURDATE()


In [14]:


df = pd.read_sql_query(sql_query, engine)

print(df.head())

Empty DataFrame
Columns: [per_no, sec_cd]
Index: []
